In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm.auto import tqdm
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys,psutil
from collections import Counter,deque
import itertools
import os

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

In [212]:
sample_cnt = 100
x=np.random.randint(low=0,high=9,size=(sample_cnt,2))
y=x[:,0]*x[:,1]
samples = np.hstack((x,y.reshape(sample_cnt,-1)))
samples[:10]

array([[ 7,  0,  0],
       [ 2,  4,  8],
       [ 8,  3, 24],
       [ 7,  6, 42],
       [ 1,  3,  3],
       [ 8,  5, 40],
       [ 4,  6, 24],
       [ 3,  2,  6],
       [ 4,  7, 28],
       [ 3,  1,  3]])

In [216]:
item2idx=dict([(str(i),i) for i in range(0,10)])
for item in samples[2:3]:
    for number in item:
        [item2idx[char] for char in str(number)]
            

[8]

[3]

[2, 4]

In [176]:
class Calc(tf.keras.Model):
    def __init__(self,total_size=100,emb_dim=3,**kwargs):
        super().__init__(name="M",**kwargs)
        self.emb = tf.keras.layers.Embedding(total_size, emb_dim)
#         self.emb = tf.keras.layers.Embedding(total_size, emb_dim)
#         self.RNN = tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform')
        self.lstm = tf.keras.layers.LSTM(16, return_sequences=False)
        self.prediction = tf.keras.layers.Dense(total_size)
        
    def call(self, inp, training=False):
        x = self.emb(inp)
        x = self.lstm(x)
        x = self.prediction(x)
        return x
    
M = Calc(total_size=10)
# M.build((1,4))
M(tf.zeros((1,10)))
M.summary()


M.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=tf.losses.MSE, metrics=['acc'])



<tf.Tensor: id=30340, shape=(1, 10), dtype=float32, numpy=
array([[-0.0270931 , -0.0144291 ,  0.00920377,  0.00231422, -0.01453765,
         0.00457465, -0.02392171, -0.01320602,  0.00081663,  0.00847211]],
      dtype=float32)>

Model: "M"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     multiple                  30        
_________________________________________________________________
lstm_88 (LSTM)               multiple                  1280      
_________________________________________________________________
dense_55 (Dense)             multiple                  170       
Total params: 1,480
Trainable params: 1,480
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = keras.Sequential([
        layers.Embedding(input_dim=30000, output_dim=32, input_length=maxlen),
        layers.LSTM(32, return_sequences=True),
        layers.LSTM(1, activation='sigmoid', return_sequences=False)
    ])
model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])

In [105]:
np.random.seed(9)
vocab_size = 10 # 所有词有多少个
seq_length = 3 # 一个句子里词的个数 或者 一个样本的特征个数
emb_dim = 3     # 每个词的词向量维数 或者 一个特征的隐向量维数
batch_size = 2  # 一个输入batch大小 | inp -> emb: (batch_size, seq_length) -> (batch_size, seq_length, emb_dim)
lstm_hidden_size = 16

input_arr = np.random.randint(vocab_size, size=(batch_size, seq_length))
input_arr.shape
input_arr

"embl"
embl=tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=emb_dim)#, input_length=seq_length)
embl(input_arr).numpy().shape
embl(input_arr).numpy()

"densel"
densel=tf.keras.layers.Dense(units=4)
densel(embl(input_arr)).numpy().shape
densel(embl(input_arr)).numpy()
"densel kernel bias"
densel.kernel.shape
densel.bias.shape


"flattenl"
flattenl=tf.keras.layers.Flatten()
flattenl(embl(input_arr)).numpy().shape
flattenl(embl(input_arr)).numpy()

"densel(flattenl)"
densel=tf.keras.layers.Dense(units=4)
densel(flattenl(embl(input_arr))).numpy().shape
densel(flattenl(embl(input_arr))).numpy()
"densel(flattenl) kernel bias"
densel.kernel.shape
densel.bias.shape

"lstml0"
lstml0=tf.keras.layers.LSTM(lstm_hidden_size, return_sequences=True)
lstml0(embl(input_arr)).numpy().shape
lstml0(embl(input_arr)).numpy()

"lstml1"
lstml1=tf.keras.layers.LSTM(lstm_hidden_size, return_sequences=False)
lstml1(embl(input_arr)).numpy().shape
lstml1(embl(input_arr)).numpy()
# lstml1=tf.keras.layers.LSTM(1, activation='sigmoid', return_sequences=False)
# lstml0.build((batch_size))
# lstml0.count_params()

"lstml2"
lstml2=tf.keras.layers.LSTM(lstm_hidden_size, return_sequences=True, return_state=True)
output,h,c=[i.numpy() for i in lstml2(embl(input_arr))]
[i.numpy().shape for i in lstml2(embl(input_arr))]
f"lstml2|output {output.shape}"
output
f"lstml2|h {h.shape}"
h
f"lstml2|c {c.shape}"
c
f"lstml2|output[:,-1,:] {output[:,-1,:].shape}"
"output[:,-1,:] 就是 h"
output[:,-1,:]

(2, 3)

array([[5, 6, 8],
       [6, 1, 6]])

'embl'

(2, 3, 3)

array([[[ 0.00707006, -0.04910037,  0.03553182],
        [ 0.02090341, -0.02885584, -0.0459761 ],
        [-0.03287369, -0.00508959, -0.04202091]],

       [[ 0.02090341, -0.02885584, -0.0459761 ],
        [-0.03563143,  0.02602991, -0.00854288],
        [ 0.02090341, -0.02885584, -0.0459761 ]]], dtype=float32)

...

array([[ 1.4377554e-03, -2.7362090e-03,  2.1980060e-03, -5.3468780e-03,
        -2.3645917e-03, -3.8110735e-03, -4.2115565e-04, -2.3473583e-03,
        -7.8789415e-03,  3.4232896e-03,  1.3597072e-03, -1.0605084e-03,
        -5.1682354e-03, -5.1462403e-03, -1.6841151e-04,  8.4685470e-04],
       [ 1.7602793e-03, -2.7579349e-03, -1.1067605e-05, -3.2701129e-03,
        -2.1379753e-03, -3.6304409e-04,  3.2441649e-03, -8.4491994e-04,
        -5.3220950e-03,  1.1092456e-03, -2.2108259e-03, -1.6482201e-03,
        -3.5803975e-03, -5.6694294e-03,  2.2526544e-03,  4.1536954e-03]],
      dtype=float32)

'lstml2|c (2, 16)'

array([[ 2.8641289e-03, -5.4758484e-03,  4.3913056e-03, -1.0680488e-02,
        -4.7426727e-03, -7.6716114e-03, -8.3775475e-04, -4.7097057e-03,
        -1.5849177e-02,  6.7879874e-03,  2.7233588e-03, -2.1178389e-03,
        -1.0246087e-02, -1.0314950e-02, -3.3663772e-04,  1.6894371e-03],
       [ 3.5152622e-03, -5.5151461e-03, -2.2198772e-05, -6.5621305e-03,
        -4.3068398e-03, -7.2770193e-04,  6.4515630e-03, -1.6977179e-03,
        -1.0668411e-02,  2.2096748e-03, -4.4574859e-03, -3.2761570e-03,
        -7.1199415e-03, -1.1275336e-02,  4.5020487e-03,  8.2821259e-03]],
      dtype=float32)

'lstml2|output[:,-1,:] (2, 16)'

array([[ 1.4377554e-03, -2.7362090e-03,  2.1980060e-03, -5.3468780e-03,
        -2.3645917e-03, -3.8110735e-03, -4.2115565e-04, -2.3473583e-03,
        -7.8789415e-03,  3.4232896e-03,  1.3597072e-03, -1.0605084e-03,
        -5.1682354e-03, -5.1462403e-03, -1.6841151e-04,  8.4685470e-04],
       [ 1.7602793e-03, -2.7579349e-03, -1.1067605e-05, -3.2701129e-03,
        -2.1379753e-03, -3.6304409e-04,  3.2441649e-03, -8.4491994e-04,
        -5.3220950e-03,  1.1092456e-03, -2.2108259e-03, -1.6482201e-03,
        -3.5803975e-03, -5.6694294e-03,  2.2526544e-03,  4.1536954e-03]],
      dtype=float32)

In [161]:
np.all(lstml1.cell.bias.numpy()==lstml1.get_weights()[2])
np.all(lstml1.cell.recurrent_kernel.numpy() == lstml1.get_weights()[1])
np.all(lstml1.cell.kernel.numpy() == lstml1.get_weights()[0])
[w.shape for w in lstml1.get_weights()]
"cell.kernel, cell.recurrent_kernel, cell.bias"
lstml1.count_params()

True

True

True

[(3, 64), (16, 64), (64,)]

'cell.kernel, cell.recurrent_kernel, cell.bias'

1280

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model